# Initialize dependencies and get data

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

In [2]:
# https://storage.googleapis.com/tensorflow/tf-keras-datasets/mnist.npz

# the data, shuffled and split between train and test sets
# (x_train, y_train), (x_test, y_test) = mnist.load_data()

import numpy as np

path = "./datasets/mnist.npz"
with np.load(path, allow_pickle=True) as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

In [3]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

# Pre-process image data

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 784)
60000 train samples
10000 test samples


In [8]:
x_test.shape

(10000, 28, 28, 1)

In [10]:
# x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])
print(x_train.shape, x_test.shape)

(60000, 784) (10000, 784)


In [11]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=250, multi_class="ovr")

In [19]:
model.fit(x_train, y_train)

c:\Users\karim\Documents\GitHub\practical-machine-learning-with-python\.venv\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [17]:
model.predict_proba(x_test[:1, :])

array([[1.00649753e-06, 8.44033007e-13, 2.36543423e-06, 1.94093810e-03,
        9.88542776e-08, 1.19763848e-05, 1.37259102e-11, 9.97773986e-01,
        9.86489552e-06, 2.59763805e-04]])

In [18]:
model.predict(x_test[:1, :])

array([7], dtype=uint8)

In [14]:
model.score(x_test, y_test), model.score(x_train, y_train)

(0.9256, 0.9346166666666667)

In [5]:
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# Build a CNN based deep neural network

In [12]:
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(20, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(15, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Visualize the network architecture

In [13]:
# from IPython.display import SVG
# from keras.utils.vis_utils import model_to_dot

# SVG(model_to_dot(model, show_shapes=True, 
#                  show_layer_names=True, rankdir='TB').create(prog='dot', format='svg'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 16)        160       
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 20)        2900      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 20)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 20)        0         
                                                                 
 flatten_1 (Flatten)         (None, 2880)              0         
                                                                 
 dense_2 (Dense)             (None, 15)                43215     
                                                      

# Build the model

In [14]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [15]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=2,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/2
469/469 [==============================] - 66s 139ms/step - loss: 2.2986 - accuracy: 0.1181 - val_loss: 2.2924 - val_accuracy: 0.1133
Epoch 2/2
469/469 [==============================] - 59s 125ms/step - loss: 2.2922 - accuracy: 0.1246 - val_loss: 2.2857 - val_accuracy: 0.1168


# Predict and test model performance

In [16]:
score = model.evaluate(x_test, y_test, verbose=1)

313/313 [==============================] - 4s 12ms/step - loss: 2.2857 - accuracy: 0.1168


In [17]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 2.285707712173462
Test accuracy: 0.11680000275373459


In [18]:
x_test.shape

(10000, 28, 28, 1)

In [23]:
y_t = model.predict(x_test[0:3])

1/1 [==============================] - 0s 97ms/step


In [25]:
np.argmax(y_t[0])

0